In [ ]:
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
import numpy as np
from typing import Callable, Any, List

search_space = [ # note: 'name' is used directly as a kwarg
    Real(1e-5, 1e-3, name='delta', prior='log-uniform'),
    Real(0.5, 2.0, name='obs_cov_reg', prior='log-uniform'),
    Real(0.01, 0.1, name='trans_cov_avg', prior='log-uniform'),
    Real(0.1, 1.0, name='obs_cov_avg', prior='log-uniform')
]

def bayesian_optimize_workflow(execute_workflow_fn: Callable, pairs_timeseries_data: pd.Series, search_space: List[Real]):
    param_names = [dim.name for dim in search_space]
    
    # objective function to minimize
    @use_named_args(search_space)
    def objective(**params):
        output = execute_workflow_fn(
            pairs_timeseries_data,
            **params,
            verbose=False  # for speed
        )
        val_mse = output['val_mse']
        return val_mse

    # run Bayesian optimization
    res = gp_minimize(
        func=objective,
        dimensions=search_space,
        n_calls=30,           # Number of evaluations of execute_kalman_workflow
        n_random_starts=10,   # Start with 10 random points before fitting a GP
        random_state=42
    )

    # print or save best hyperparameters
    best_params = {
        'delta': res.x[0],
        'obs_cov_reg': res.x[1],
        'trans_cov_avg': res.x[2],
        'obs_cov_avg': res.x[3]
    }
    print("Best hyperparameters:", best_params)
    print("Best validation MSE:", res.fun)
    